In [ ]:
%%writefile app.py
import yfinance as yf
import streamlit as st
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Bidirectional,Conv1D, MaxPooling1D, Flatten,SimpleRNN,GRU
from datetime import datetime
import warnings
from statsmodels.tsa.arima.model import ARIMA
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from keras.optimizers import SGD
from sklearn.model_selection import GridSearchCV


lstm_predictions = []
bilstm_predictions = []
cnn_predictions = []
rnn_predictions = []
gru_predictions = []
xgb_predictions = []

sc = MinMaxScaler(feature_range=(0, 1))



def get_stock_data(ticker, start_date, end_date):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    return stock_data


def prepare_data(dataset_train):
    # Verileri ölçeklendirme
    training_set_scaled = sc.fit_transform(dataset_train.iloc[:, 4:5].values)
    # Veri yapısını oluşturma
    X_train = []
    y_train = []
    for i in range(60, len(training_set_scaled)):
        X_train.append(training_set_scaled[i-60:i, 0])
        y_train.append(training_set_scaled[i, 0])
    X_train, y_train = np.array(X_train), np.array(y_train)

    # Yeniden şekillendirme
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

    return X_train, y_train


st.title('Borsa Kapanış Fiyatı Tahmini')

# Data Preprocessing
st.header('Veri Ön İşleme')

# Define ticker symbol and date range
ticker_symbol = st.selectbox("Hisse Sembolünü Seçiniz.", ["AMZN", "AAPL", "GOOGL", "MSFT"])
start_date = st.date_input("Başlangıç Tarihi", value=pd.to_datetime('2020-01-01'))
end_date = st.date_input("Bitiş Tarihi", value=pd.to_datetime('2024-01-01'))

# Slider for selecting percentage of data for training set
train_percentage = st.slider("Eğitim İçin Yüzde Ayarlayın", 20, 100, 80)

epoch = st.number_input("Epoch", min_value=0)

cboxLSTM = st.checkbox('LSTM')
cboxBILSTM = st.checkbox('BILSTM')
# cboxARIMA = st.checkbox('ARIMA')
cboxCNN = st.checkbox('CNN')
cboxRNN = st.checkbox('RNN')
cboxGRU = st.checkbox('GRU')
cboxXGB = st.checkbox('XGBoost')

# Load stock data when button is clicked
if st.button("Verileri Al ve Eğitmeye Başla"):
    stock_data = get_stock_data(ticker_symbol, start_date, end_date)
    st.subheader('Borsa Verileri')
    st.write(stock_data)

    # Calculate number of rows for training set based on selected percentage
    total_rows = len(stock_data)
    train_rows = int(total_rows * train_percentage / 100)

    # Create training and test data sets
    training_data = stock_data.head(train_rows)
    test_data = stock_data.tail(total_rows - train_rows)

    st.subheader('Eğitim Veriseti - Borsa Verisi')
    st.write(training_data)

    st.subheader('Test Veriseti - Borsa Verisi')
    st.write(test_data)

    test_data.reset_index(inplace=True)

    date_df = pd.DataFrame(test_data.iloc[:, 0].values, columns=['Date'])


    # Load dataset_train globally
    dataset_train = training_data

    # Fitting the LSTM to the Training set
    with st.spinner('Model eğitiliyor...'):
        # Making predictions
        st.header('Tahmin Yapılıyor')
        if cboxLSTM :
           # Initialising the LSTM
          with st.spinner('LSTM modeli eğitiliyor...'):
            X_train_lstm, y_train_lstm = prepare_data(dataset_train)
            regressorLSTM = Sequential()

            # Adding the LSTM layers with Dropout regularisation
            regressorLSTM.add(LSTM(units=50, return_sequences=True, input_shape=(X_train_lstm.shape[1], 1)))
            regressorLSTM.add(Dropout(0.2))

            regressorLSTM.add(LSTM(units=50, return_sequences=True))
            regressorLSTM.add(Dropout(0.2))

            regressorLSTM.add(LSTM(units=50, return_sequences=True))
            regressorLSTM.add(Dropout(0.2))

            regressorLSTM.add(LSTM(units=50))
            regressorLSTM.add(Dropout(0.2))

            # Adding the output layer
            regressorLSTM.add(Dense(units=1))

            # Compiling the RNN
            regressorLSTM.compile(optimizer='adam', loss='mean_squared_error')
            regressorLSTM.fit(X_train_lstm, y_train_lstm, epochs=epoch, batch_size=32)

            # Getting the real stock price of 2017
            dataset_test = test_data
            real_stock_price = dataset_test.iloc[:, 4:5].values

            # Getting the predicted stock price of 2017
            dataset_total = pd.concat((dataset_train['Close'], dataset_test['Close']), axis=0)
            inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
            inputs = inputs.reshape(-1, 1)
            inputs = sc.transform(inputs)
            X_test = []
            for i in range(60, len(inputs)):
                X_test.append(inputs[i-60:i, 0])
            X_test = np.array(X_test)
            X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
            predicted_stock_price_lstm = regressorLSTM.predict(X_test)
            predicted_stock_price_lstm = sc.inverse_transform(predicted_stock_price_lstm)

            st.header('LSTM Sonuçları')
            plt.plot(real_stock_price, color='red', label='Gerçek '+ ticker_symbol +' Kapanış Fiyatları')
            plt.plot(predicted_stock_price_lstm, color='blue', label='Tahmin Edilen '+ ticker_symbol +' Kapanış Fiyatları')
            plt.title('LSTM Model Tahminleri')
            plt.xlabel('Zaman')
            plt.ylabel('Kapanış Fiyatı')
            plt.legend()
            plt.show()
            st.pyplot(plt)
            df_lstm = pd.DataFrame(predicted_stock_price_lstm, columns=['Tahmin Edilen LSTM Sonuçları'])
            lstm_predictions = predicted_stock_price_lstm.flatten()
            st.header('Tahmin Edilen ' + ticker_symbol + ' Hissesi Fiyatları LSTM - Tablo')
            df_lstm = pd.concat([date_df, df_lstm], axis=1)
            df_lstm = df_lstm.set_index('Date')
            df_lstm.index.name = 'Tarih'
            st.write(df_lstm)


        if cboxBILSTM :
          with st.spinner('BILSTM modeli eğitiliyor...'):
            X_train_bilstm, y_train_bilstm = prepare_data(dataset_train)
            # Initialising the RNN
            regressorBILSTM = Sequential()

            # Adding the first Bidirectional LSTM layer and some Dropout regularisation
            regressorBILSTM.add(Bidirectional(LSTM(units=50, return_sequences=True), input_shape=(X_train_bilstm.shape[1], 1)))
            regressorBILSTM.add(Dropout(0.2))

            # Adding a second Bidirectional LSTM layer and some Dropout regularisation
            regressorBILSTM.add(Bidirectional(LSTM(units=50, return_sequences=True)))
            regressorBILSTM.add(Dropout(0.2))

            # Adding a third Bidirectional LSTM layer and some Dropout regularisation
            regressorBILSTM.add(Bidirectional(LSTM(units=50, return_sequences=True)))
            regressorBILSTM.add(Dropout(0.2))

            # Adding a fourth Bidirectional LSTM layer and some Dropout regularisation
            regressorBILSTM.add(Bidirectional(LSTM(units=50)))
            regressorBILSTM.add(Dropout(0.2))

            # Adding the output layer
            regressorBILSTM.add(Dense(units=1))

            # Compiling the RNN
            regressorBILSTM.compile(optimizer='adam', loss='mean_squared_error')

            # Fitting the RNN to the Training set
            history = regressorBILSTM.fit(X_train_bilstm, y_train_bilstm, epochs=epoch, batch_size=32)

            dataset_test = test_data
            real_stock_price = dataset_test.iloc[:, 4:5].values

            # Part 3 - Making the predictions and visualising the results
            dataset_total = pd.concat((dataset_train['Close'], dataset_test['Close']), axis=0)
            inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
            inputs = inputs.reshape(-1, 1)
            inputs = sc.transform(inputs)
            X_test = []
            for i in range(60, len(inputs)):
                X_test.append(inputs[i-60:i, 0])
            X_test = np.array(X_test)
            X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

            predicted_stock_price_bilstm = regressorBILSTM.predict(X_test)
            predicted_stock_price_bilstm = sc.inverse_transform(predicted_stock_price_bilstm)

            st.header('BILSTM Sonuçları')
            plt.clf()
            plt.plot(real_stock_price, color='red', label='Gerçek '+ ticker_symbol +' Kapanış Fiyatları')
            plt.plot(predicted_stock_price_bilstm, color='blue', label='Tahmin Edilen '+ ticker_symbol +' Kapanış Fiyatları')
            plt.title('BILSTM Model Tahminleri')
            plt.xlabel('Zaman')
            plt.ylabel('Kapanış Fiyatı')
            plt.legend()
            plt.show()
            st.pyplot(plt)
            df_bilstm = pd.DataFrame(predicted_stock_price_bilstm, columns=['Tahmin Edilen BiLSTM Sonuçları'])
            bilstm_predictions = predicted_stock_price_bilstm.flatten()
            st.header('Tahmin Edilen ' + ticker_symbol + ' Hissesi Fiyatları BILSTM - Tablo')
            df_bilstm = pd.concat([date_df, df_bilstm], axis=1)
            df_bilstm = df_bilstm.set_index('Date')
            df_bilstm.index.name = 'Tarih'
            st.write(df_lstm)

        if cboxCNN :
          with st.spinner('CNN modeli eğitiliyor...'):
            X_train_cnn, y_train_cnn = prepare_data(dataset_train)
            # CNN modelini oluşturalım
            modelCNN = Sequential()

            # Birinci evrişim katmanını ekleyelim
            modelCNN.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(60, 1)))

            # Birinci havuzlama katmanını ekleyelim
            modelCNN.add(MaxPooling1D(pool_size=2))

            # İkinci evrişim katmanını ekleyelim
            modelCNN.add(Conv1D(filters=64, kernel_size=3, activation='relu'))

            # İkinci havuzlama katmanını ekleyelim
            modelCNN.add(MaxPooling1D(pool_size=2))

            # Düzleştirme katmanını ekleyelim
            modelCNN.add(Flatten())

            # Tam bağlı (fully connected) katmanı ekleyelim
            modelCNN.add(Dense(units=50, activation='relu'))

            # Çıkış katmanını ekleyelim
            modelCNN.add(Dense(units=1))

            # ModelCNNi derleyelim
            modelCNN.compile(optimizer='adam', loss='mean_squared_error')

            # ModelCNNi eğitelim
            modelCNN.fit(X_train_cnn, y_train_cnn, epochs=epoch, batch_size=32)

            # Test verisi üzerinde tahminlerde bulunalım
            dataset_test = test_data
            real_stock_price = dataset_test.iloc[:, 4:5].values
            dataset_total = pd.concat((dataset_train['Close'], dataset_test['Close']), axis=0)
            inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
            inputs = inputs.reshape(-1, 1)
            inputs = sc.transform(inputs)
            X_test = []
            for i in range(60, len(inputs)):
                X_test.append(inputs[i-60:i, 0])
            X_test = np.array(X_test)
            X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
            predicted_stock_price_cnn = modelCNN.predict(X_test)
            predicted_stock_price_cnn = sc.inverse_transform(predicted_stock_price_cnn)

            # Sonuçları görselleştirelim
            st.header('CNN Sonuçları')
            plt.clf()
            plt.plot(real_stock_price, color='red',label='Gerçek '+ ticker_symbol +' Kapanış Fiyatları')
            plt.plot(predicted_stock_price_cnn, color='blue', label='Tahmin Edilen '+ ticker_symbol +' Kapanış Fiyatları')
            plt.title('CNN Model Tahminleri')
            plt.xlabel('Zaman')
            plt.ylabel('Kapanış Fiyatı')
            plt.legend()
            plt.show()
            st.pyplot(plt)
            df_cnn = pd.DataFrame(predicted_stock_price_cnn, columns=['Tahmin Edilen CNN Sonuçları'])
            cnn_predictions = predicted_stock_price_cnn.flatten()
            st.header('Tahmin Edilen ' + ticker_symbol + ' Hissesi Fiyatları CNN - Tablo')
            df_cnn = pd.concat([date_df, df_cnn], axis=1)
            df_cnn = df_cnn.set_index('Date')
            df_cnn.index.name = 'Tarih'
            st.write(df_cnn)

        if cboxRNN :
          with st.spinner('RNN modeli eğitiliyor...'):
            X_train_rnn, y_train_rnn = prepare_data(dataset_train)

            # RNN modelini oluşturalım
            modelRNN = Sequential()

            # RNN katmanını ekleyelim
            modelRNN.add(SimpleRNN(units=50, input_shape=(60, 1)))
            modelRNN.add(Dropout(0.2))

            # Tam bağlı (fully connected) katmanı ekleyelim
            modelRNN.add(Dense(units=1))
            modelRNN.add(Dropout(0.2))

            # modelRNNi derleyelim
            modelRNN.compile(optimizer='adam', loss='mean_squared_error')

            # modelRNNi eğitelim
            modelRNN.fit(X_train_rnn, y_train_rnn, epochs=epoch, batch_size=32)

            # Test verisi üzerinde tahminlerde bulunalım
            dataset_test = test_data
            real_stock_price = dataset_test.iloc[:, 4:5].values

            dataset_total = pd.concat((dataset_train['Close'], dataset_test['Close']), axis=0)
            inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
            inputs = inputs.reshape(-1, 1)
            inputs = sc.transform(inputs)
            X_test = []

            for i in range(60, len(inputs)):
                X_test.append(inputs[i-60:i, 0])
            X_test = np.array(X_test)
            X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
            predicted_stock_price_rnn = modelRNN.predict(X_test)
            predicted_stock_price_rnn = sc.inverse_transform(predicted_stock_price_rnn)

            # Sonuçları görselleştirelim
            st.header('RNN Sonuçları')
            plt.clf()
            plt.plot(real_stock_price, color='red', label='Gerçek '+ ticker_symbol +' Kapanış Fiyatları')
            plt.plot(predicted_stock_price_rnn, color='blue', label='Tahmin Edilen '+ ticker_symbol +' Kapanış Fiyatları')
            plt.title('RNN Model Tahminleri')
            plt.xlabel('Zaman')
            plt.ylabel('Kapanış Fiyatı')
            plt.legend()
            plt.show()
            st.pyplot(plt)
            df_rnn = pd.DataFrame(predicted_stock_price_rnn, columns=['Tahmin Edilen RNN Sonuçları'])
            rnn_predictions = predicted_stock_price_rnn.flatten()
            st.header('Tahmin Edilen ' + ticker_symbol + ' Hissesi Fiyatları RNN - Tablo')
            df_rnn = pd.concat([date_df, df_rnn], axis=1)
            df_rnn = df_rnn.set_index('Date')
            df_rnn.index.name = 'Tarih'
            st.write(df_rnn)

        if cboxGRU :
          with st.spinner('GRU modeli eğitiliyor...'):
            X_train_gru, y_train_gru = prepare_data(dataset_train)

             # Getting the real stock price of 2017
            dataset_test = test_data
            real_stock_price = dataset_test.iloc[:, 4:5].values

            # Getting the predicted stock price of 2017
            dataset_total = pd.concat((dataset_train['Close'], dataset_test['Close']), axis=0)
            inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
            inputs = inputs.reshape(-1, 1)
            inputs = sc.transform(inputs)

            X_test = []
            for i in range(60, len(inputs)):
                X_test.append(inputs[i-60:i, 0])
            X_test = np.array(X_test)
            X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))


            regressorGRU = Sequential()
            # First GRU layer with Dropout regularisation
            regressorGRU.add(GRU(units=50, return_sequences=True, input_shape=(X_train_gru.shape[1],1), activation='tanh'))
            regressorGRU.add(Dropout(0.2))
            # Second GRU layer
            regressorGRU.add(GRU(units=50, return_sequences=True, input_shape=(X_train_gru.shape[1],1), activation='tanh'))
            regressorGRU.add(Dropout(0.2))
            # Third GRU layer
            regressorGRU.add(GRU(units=50, return_sequences=True, input_shape=(X_train_gru.shape[1],1), activation='tanh'))
            regressorGRU.add(Dropout(0.2))
            # Fourth GRU layer
            regressorGRU.add(GRU(units=50, activation='tanh'))
            regressorGRU.add(Dropout(0.2))
            # The output layer
            regressorGRU.add(Dense(units=1))

            regressorGRU.compile(optimizer=SGD(learning_rate=0.01, momentum=0.9, nesterov=False), loss='mean_squared_error')

            # fitting the model

            regressorGRU.fit(X_train_gru, y_train_gru, epochs=epoch, batch_size=150)

            predicted_stock_price_gru = regressorGRU.predict(X_test)
            predicted_stock_price_gru = sc.inverse_transform(predicted_stock_price_gru)

            st.header('GRU Sonuçları')
            plt.clf()
            plt.plot(real_stock_price, color='red', label='Gerçek '+ ticker_symbol +' Kapanış Fiyatları')
            plt.plot(predicted_stock_price_gru, color='blue', label='Tahmin Edilen '+ ticker_symbol +' Kapanış Fiyatları')
            plt.title('GRU Model Tahminleri')
            plt.xlabel('Zaman')
            plt.ylabel('Kapanış Fiyatı')
            plt.legend()
            plt.show()
            st.pyplot(plt)
            df_gru = pd.DataFrame(predicted_stock_price_gru, columns=['Tahmin Edilen GRU Sonuçları'])
            gru_predictions = predicted_stock_price_gru.flatten()
            st.header('Tahmin Edilen ' + ticker_symbol + ' Hissesi Fiyatları GRU - Tablo')
            df_gru = pd.concat([date_df, df_gru], axis=1)
            df_gru = df_gru.set_index('Date')
            df_gru.index.name = 'Tarih'
            st.write(df_gru)

        if cboxXGB:
           with st.spinner('XGBoost modeli eğitiliyor...'):

              X_train_xgb = []
              y_train_xgb = dataset_train['Close'].values[60:]
              for i in range(60, len(dataset_train)):
                  features = []
                  # Özellik 1: Kapanış fiyatları
                  features.append(dataset_train['Close'].values[i-1])
                  # Özellik 2: Hacim
                  features.append(dataset_train['Volume'].values[i-1])
                  # Özellik 3: 5 günlük hareketli ortalama
                  features.append(dataset_train['Close'].rolling(window=5).mean().values[i-1])
                  # Özellik 4: 30 günlük hareketli ortalama
                  features.append(dataset_train['Close'].rolling(window=30).mean().values[i-1])
                  # Özellik 5: 365 günlük hareketli ortalama
                  features.append(dataset_train['Close'].rolling(window=365).mean().values[i-1])
                  X_train_xgb.append(features)
              X_train_xgb = np.array(X_train_xgb)

              # XGBoost veri yapısına dönüştürme
              dtrain = xgb.DMatrix(X_train_xgb, label=y_train_xgb)

              # XGBoost parametre ayarları
              param_grid = {
                  'max_depth': [3, 6, 9],
                  'learning_rate': [0.01, 0.1, 0.3],
                  'n_estimators': [100, 200, 300]
              }

              # Modeli oluşturma
              xgb_model = xgb.XGBRegressor()
              grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

              grid_search.fit(X_train_xgb, y_train_xgb)

              # En iyi parametreleri bulma
              best_params = grid_search.best_params_

              # Final modelini oluşturma
              final_xgb_model = xgb.XGBRegressor(**best_params)
              final_xgb_model.fit(X_train_xgb, y_train_xgb)

              # Test veri seti için tahminler yapma
              X_test_xgb = []
              for i in range(len(dataset_train), len(dataset_train) + len(test_data)):
                  features = []
                  features.append(stock_data['Close'].values[i-1])
                  features.append(stock_data['Volume'].values[i-1])
                  features.append(stock_data['Close'].rolling(window=5).mean().values[i-1])
                  features.append(stock_data['Close'].rolling(window=30).mean().values[i-1])
                  features.append(stock_data['Close'].rolling(window=365).mean().values[i-1])
                  X_test_xgb.append(features)
              X_test_xgb = np.array(X_test_xgb)

              predicted_stock_price_xgb = final_xgb_model.predict(X_test_xgb)
              real_stock_price = test_data.iloc[:, 4:5].values

              st.header('XGBoost Sonuçları')
              plt.clf()
              plt.plot(real_stock_price, color='red', label='Gerçek '+ ticker_symbol +' Kapanış Fiyatları')
              plt.plot(predicted_stock_price_xgb, color='blue', label='Tahmin Edilen '+ ticker_symbol +' Kapanış Fiyatları (XGBoost)')
              plt.title(ticker_symbol + ' Borsa Değeri Tahmini (XGBoost)')
              plt.xlabel('Zaman')
              plt.ylabel('Kapanış Fiyatı')
              plt.legend()
              plt.show()
              st.pyplot(plt)
              df_xgb = pd.DataFrame(predicted_stock_price_xgb, columns=['Tahmin Edilen XGBoost Sonuçları'])
              xgb_predictions = predicted_stock_price_xgb.flatten()
              st.header('Tahmin Edilen ' + ticker_symbol + ' Hissesi Fiyatları XGBoost - Tablo')
              df_xgb = pd.concat([date_df, df_xgb], axis=1)
              df_xgb = df_xgb.set_index('Date')
              df_xgb.index.name = 'Tarih'
              st.write(df_xgb)

        all_predictions = []

        if len(lstm_predictions) > 0:
            all_predictions.append(lstm_predictions)

        if len(bilstm_predictions) > 0:
            all_predictions.append(bilstm_predictions)

        # if len(arima_predictions) > 0:
        #   all_predictions.append(arima_predictions)

        if len(cnn_predictions) > 0:
            all_predictions.append(cnn_predictions)

        if len(rnn_predictions) > 0:
            all_predictions.append(rnn_predictions)

        if len(gru_predictions) > 0:
            all_predictions.append(gru_predictions)

        if len(xgb_predictions) > 0:
            all_predictions.append(xgb_predictions)



        if len(all_predictions) > 1:
          stacked_predictions = np.mean(all_predictions, axis=0)
          meta_model = LinearRegression()
          meta_model.fit(np.array(all_predictions).T, real_stock_price)  # real_stock_price, tahmin edilen fiyatların gerçek değerleridir
          stacked_predictions_test = meta_model.predict(np.array(all_predictions).T)
          plt.clf()
          st.header('STACKED ENSEMBLE Sonuçları')
          plt.plot(real_stock_price, color='red', label='Gerçek Kapanış Fiyatları')
          plt.plot(stacked_predictions_test, color='blue', label='Stacked Ensemble Tahminleri')
          plt.title('Stacked Ensemble Tahminleri')
          plt.xlabel('Zaman')
          plt.ylabel('Kapanış Fiyatı')
          plt.legend()
          plt.show()
          st.pyplot(plt)

          df_SEnsemble = pd.DataFrame(stacked_predictions_test, columns=['Tahmin Edilen Stacked Ensemble Sonuçları'])
          df_SEnsemble = pd.concat([date_df, df_SEnsemble], axis=1)
          SEnsemble_predictions = stacked_predictions_test.flatten()
          st.header('Tahmin Edilen ' + ticker_symbol + ' Hissesi Fiyatları Stacked Ensemble - Tablo')
          df_SEnsemble = df_SEnsemble.set_index('Date')
          df_SEnsemble.index.name = 'Tarih'
          st.write(df_SEnsemble)
          st.success('Model eğitimi tamamlandı!')


          # Displaying predicted prices in a table


          # # Tahmin sonuçlarını birleştirelim
          # df_combined = all_predictions[0]
          # for df in all_predictions[1:]:
          #     df_combined = pd.merge(df_combined, df, on='Date')

          # # 'Date' kolonunu indeks olarak ayarlayıp adını 'Tarih' olarak değiştirme
          # df_combined = df_combined.set_index('Date')
          # df_combined.index.name = 'Tarih'

          # # Tahmin verilerini Streamlit ile yazdırma
          # st.header('Tahmin Edilen Hisse Fiyatları')
          # st.write(df_combined)

Overwriting app.py


In [ ]:
!pip install streamlit

In [ ]:
!streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.197.120.119:8501

npx: installed 22 in 2.274s
your url is: https://brown-planes-sin.loca.lt
2024-06-02 16:57:54.747678: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-02 16:57:54.747746: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-02 16:57:54.749535: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-02 16:57:54.760086: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimiz